Các bạn sử dụng tập dữ liệu của bài tập 5, mô hình dữ liệu x theo dạng bậc 6 và làm các công việc sau:

1. Chia dữ liệu ra thành 2 tập: training (70%) và validation (30%). Phải đảm bảo việc chia dữ liệu là ngẫu nhiên và tỷ lệ positive và negative cân bằng.

2. Viết chương trình cho phép học các tham số của mô hình phân loại phi tuyến trên có sử dụng regularization L2 và L1.

3. Tính J ở mỗi vòng lặp cho cả hai tập, chọn điểm dừng phù hợp.

4. Thay đổi lambda và tính J cho mỗi lambda tương ứng cho cả hai tập. Vẽ biểu đồ quan hệ giữa J và lambda từ đó lựa chọn lambda phù hợp.

In [69]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [70]:
ex5_data = pd.read_csv('ex5data.txt', header= None).to_numpy()

In [71]:
ex5_data.shape

(118, 3)

In [72]:
x1, x2, y = ex5_data[:, 0], ex5_data[:, 1], ex5_data[:, 2]  

In [73]:
colors = np.where(y == 1, 'red', 'blue')

In [74]:
fig = go.Figure()

# Plot data points
fig.add_trace(go.Scatter(
    x=x1, 
    y=x2, 
    mode='markers', 
    marker=dict(color=colors, size=10),
    name='Data Points'
))

fig.update_xaxes(title= 'x1')
fig.update_yaxes(title= 'x2')


In [75]:
x = ex5_data[:, :2]

In [76]:
x_pos = x[y == 1]
x_neg = x[y == 0]

In [77]:
rng = np.random.default_rng()

rng.shuffle(x_pos)
rng.shuffle(x_neg)

In [78]:
len_train = int(0.7 * len(y))
len_valid = len(y) - len_train

In [79]:
x_train = np.vstack((x_pos[0: int(len_train / 2)], x_neg[0: len_train - int(len_train / 2)]))
x_valid = np.vstack((x_pos[0: int(len_train / 2)], x_neg[(len_train - int(len_train / 2))]))

In [80]:
np.random.shuffle(x_train)
np.random.shuffle(x_valid)

In [81]:
n = 0
X_mapFeature = np.zeros((118, 27))

for order in range(1, 7, 1):
    for i in range(0, order+1):

        x12 = x1**(order - i) * x2**(i)
        X_mapFeature[:, n] = x12
        n = n + 1

        # print(f'x1:^{(order - i)} - x2^{(i)}')
        # print(n)

In [82]:
X_mapFeature

array([[ 5.12670000e-02,  6.99560000e-01,  2.62830529e-03, ...,
         6.29470940e-04,  8.58939846e-03,  1.17205992e-01],
       [-9.27420000e-02,  6.84940000e-01,  8.60107856e-03, ...,
         1.89305413e-03, -1.39810280e-02,  1.03255971e-01],
       [-2.13710000e-01,  6.92250000e-01,  4.56719641e-02, ...,
         1.04882142e-02, -3.39734512e-02,  1.10046893e-01],
       ...,
       [-4.84450000e-01,  9.99270000e-01,  2.34691802e-01, ...,
         2.34007252e-01, -4.82684337e-01,  9.95627986e-01],
       [-6.33640000e-03,  9.99270000e-01,  4.01499650e-05, ...,
         4.00328554e-05, -6.31330588e-03,  9.95627986e-01],
       [ 6.32650000e-01, -3.06120000e-02,  4.00246023e-01, ...,
         3.51474517e-07, -1.70067777e-08,  8.22905998e-10]])

In [83]:
class Logistic_Regression_Multivariables:
    def __init__(self, *, number_of_feature: int) -> None:
        self.number_of_features = number_of_feature

    def normalize_vector(self, vector: np.ndarray) -> np.ndarray:
        mean = np.mean(vector)
        std = np.std(vector)
        std = np.where(std == 0, 1, std)
        return (vector - mean) / std
    
    def normalize_input(self, *, X: np.ndarray) -> tuple:
        norm_X = np.apply_along_axis(self.normalize_vector, arr=X, axis=0).reshape(-1, self.number_of_features)
        return norm_X

    def add_ones_columns(self, *, normalized_input: np.ndarray) -> np.ndarray:
        ones = np.ones(len(normalized_input)).reshape(-1, 1)
        x_add = np.hstack((ones, normalized_input))
        return x_add

    def predict(self, *, theta: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        y_pred = np.matmul(normalized_input, theta)
        y_pred = 1/(1 + np.exp(-y_pred))
        return y_pred
    
    def compute_loss(self, *, y_true: np.ndarray, y_pred: np.ndarray, lambda_: float, theta: np.ndarray) -> np.ndarray:
        m = len(y_true)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

        theta_update = np.zeros_like(theta)
        theta_update[1:] = theta[1:]
        J = np.sum(- y_true*np.log(y_pred) - (1 - y_true)*np.log(1 - y_pred)) / (m) + lambda_*np.sum(theta_update**2)/(2*m)
        return J
    
    def update_params(self, *, theta: np.ndarray, lr: float, y_pred: np.ndarray, 
                      y_true: np.ndarray, normalized_input: np.ndarray, lambda_: float) -> np.ndarray:
        m = len(y_true)
        E = y_pred - y_true
        dJ_dtheta = np.dot(normalized_input.T, E) / (m)

        theta_update = np.zeros_like(theta)
        theta_update[1:] = theta[1:]

        theta_updated = theta - lr*(dJ_dtheta +  lambda_*theta_update/m)
        return theta_updated
    
    def train(self, *, epochs: int, theta: np.ndarray, input: np.ndarray, lambda_: float, 
              output: np.ndarray, lr: float, plot_graph: False, color: list, time_delay: float) -> np.ndarray:
        
        output = output.reshape(-1, 1)
        normalized_input = self.normalize_input(X= input)
        normalized_input_with_ones = self.add_ones_columns(normalized_input= normalized_input)

        J_array = np.array([])
        
        for epoch in range(epochs):
            y_pred = self.predict(theta= theta, normalized_input= normalized_input_with_ones)

            J = self.compute_loss(y_true= output, y_pred= y_pred, lambda_= lambda_, theta= theta)
            J_array = np.append(arr= J_array, values= J)

            theta = self.update_params(theta= theta, lr= lr, y_pred= y_pred, lambda_= lambda_,
                                       y_true= output, normalized_input= normalized_input_with_ones)

            
    
        return J_array, theta

In [84]:
num_feature = X_mapFeature.shape[1]
Logistic_Regression = Logistic_Regression_Multivariables(number_of_feature= num_feature)

normalized_input = Logistic_Regression.normalize_input(X= X_mapFeature)
normalized_input_with_ones = Logistic_Regression.add_ones_columns(normalized_input= normalized_input)


In [85]:
np.random.seed(1)

theta_init = np.random.rand(num_feature + 1, 1)
theta_init

array([[4.17022005e-01],
       [7.20324493e-01],
       [1.14374817e-04],
       [3.02332573e-01],
       [1.46755891e-01],
       [9.23385948e-02],
       [1.86260211e-01],
       [3.45560727e-01],
       [3.96767474e-01],
       [5.38816734e-01],
       [4.19194514e-01],
       [6.85219500e-01],
       [2.04452250e-01],
       [8.78117436e-01],
       [2.73875932e-02],
       [6.70467510e-01],
       [4.17304802e-01],
       [5.58689828e-01],
       [1.40386939e-01],
       [1.98101489e-01],
       [8.00744569e-01],
       [9.68261576e-01],
       [3.13424178e-01],
       [6.92322616e-01],
       [8.76389152e-01],
       [8.94606664e-01],
       [8.50442114e-02],
       [3.90547832e-02]])

In [86]:
learning_rate_ = [0.01, 0.001, 0.003, 0.3, 0.04, 0.1]
J = np.zeros((1000, ))

# for i in learning_rate_:
J_arr, theta_arr = Logistic_Regression.train(epochs= 1000, theta= theta_init, 
                                        input= X_mapFeature, output= y, lr= 0.3, plot_graph= False,
                                        color= colors, time_delay= 0.01, lambda_= 1000)


C:\Users\Admin\AppData\Local\Temp\ipykernel_20988\3935322790.py:22: RuntimeWarning:

overflow encountered in exp

C:\Users\Admin\AppData\Local\Temp\ipykernel_20988\3935322790.py:32: RuntimeWarning:

overflow encountered in scalar multiply

c:\Users\Admin\anaconda3\envs\env_gpu\lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning:

overflow encountered in reduce

C:\Users\Admin\AppData\Local\Temp\ipykernel_20988\3935322790.py:32: RuntimeWarning:

overflow encountered in square



In [99]:
# Code test 

# Test normalized input
def normalize_vector(vector: np.ndarray) -> np.ndarray:
    mean = np.mean(vector)
    std = np.std(vector)
    std = np.where(std == 0, 1, std)
    return (vector - mean) / std

vector = np.array([[1, 2, 3, 4, 5],
                  [2, 2, 3, 4, 5],
                  [1, 2, 3, 4, 5],
                  [1, 2, 3, 4, 5]])


data_norm = np.apply_along_axis(normalize_vector, arr= vector, axis=0).reshape(-1, 5)
print(f'\n data_norm: \n {data_norm}')

# Test add_ones_columns
def add_ones_columns(normalized_input: np.ndarray) -> np.ndarray:
    ones = np.ones(len(normalized_input)).reshape(-1, 1)
    x_add = np.hstack((ones, normalized_input))
    return x_add

data_norm_add_ones_columns = add_ones_columns(normalized_input= data_norm)
print(f'\n data_norm add_ones_columns: \n {data_norm_add_ones_columns}')

# Test predict
def predict(theta: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
    y_pred = np.matmul(normalized_input, theta)
    y_pred = 1/(1 + np.exp(-y_pred))
    return y_pred

theta = np.random.randn(data_norm.shape[1] + 1, 1)
y_pred = predict(theta= theta, normalized_input= data_norm_add_ones_columns)

print(f'\n theta: \n {theta}')
print(f'\n y_pred: \n {y_pred}')

# Test compute_loss
def compute_loss(y_true: np.ndarray, y_pred: np.ndarray, lambda_: float, theta: np.ndarray) -> np.ndarray:
    m = len(y_true)
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

    theta_update = np.zeros_like(theta)
    theta_update[1:] = theta[1:]
    J = np.sum(- y_true*np.log(y_pred) - (1 - y_true)*np.log(1 - y_pred)) / (m) + lambda_*np.sum(theta_update**2)/(2*m)
    return J

y_true = np.array([[0], [0], [1], [1]])
J = compute_loss(y_true= y_true, y_pred= y_pred, lambda_= 1000, theta= theta)

print(f'\n y_true: \n {y_true}')
print(f'\n J: \n {J}')



 data_norm: 
 [[-0.57735027  0.          0.          0.          0.        ]
 [ 1.73205081  0.          0.          0.          0.        ]
 [-0.57735027  0.          0.          0.          0.        ]
 [-0.57735027  0.          0.          0.          0.        ]]

 data_norm add_ones_columns: 
 [[ 1.         -0.57735027  0.          0.          0.          0.        ]
 [ 1.          1.73205081  0.          0.          0.          0.        ]
 [ 1.         -0.57735027  0.          0.          0.          0.        ]
 [ 1.         -0.57735027  0.          0.          0.          0.        ]]

 theta: 
 [[ 0.61720311]
 [ 0.30017032]
 [-0.35224985]
 [-1.1425182 ]
 [-0.34934272]
 [-0.20889423]]

 y_pred: 
 [[0.60918786]
 [0.75714925]
 [0.60918786]
 [0.60918786]]

 y_true: 
 [[0]
 [0]
 [1]
 [1]]

 J: 
 211.48741674978447
